In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from shapely.ops import nearest_points

pd.set_option('display.max_columns', None)

df_zonas=pd.read_excel("dados/tabela_regioes_sp.xlsx")

Distrito censitário e zona ao qual a parada pertence

In [2]:

gdf_setores_censitarios = gpd.read_file("dados/POPULAÇÃO-CENSO-2022-IBGE/BR_setores_CD2022.zip")

cols_v = [f'v000{i}' for i in range(1, 8)]
for col in cols_v:
    gdf_setores_censitarios[col] = pd.to_numeric(gdf_setores_censitarios[col], errors='coerce')

gdf_setores_censitarios.to_parquet("dados/setores_censitarios_2022.parquet")

In [3]:
df_paradas = pd.read_parquet("dados/paradas_historico.parquet").drop_duplicates()
gdf_paradas = gpd.GeoDataFrame(
    df_paradas,
    geometry=[Point(xy) for xy in zip(df_paradas['stop_lon'], df_paradas['stop_lat'])],
    crs='EPSG:4326'
)

gdf_paradas = gdf_paradas.to_crs(gdf_setores_censitarios.crs)

In [4]:

# Spatial join: cada parada recebe dados do bairro em que está
gdf_resultado = gpd.sjoin(
    gdf_paradas,
    gdf_setores_censitarios,
    how='left',
    predicate='within'
).drop(columns=['index_right'])


In [5]:
#adiciona zona, subprefeitura
gdf_resultado=gdf_resultado.merge(df_zonas,left_on='NM_DIST',right_on='Distritos')

KeyError: 'NM_DIST'

Estação de metro e trem mais próxima

In [ ]:
def load_estacoes(path):
    gdf = gpd.read_file(path)
    if 'geometry' not in gdf.columns or gdf.geometry.is_empty.any():
        raise ValueError(f'Arquivo {path} sem geometria válida.')
    gdf = gdf.to_crs('EPSG:31983')  # CRS métrico para distância em metros
    return gdf

In [ ]:
gdf_resultado = gdf_resultado.to_crs('EPSG:31983')

In [ ]:
# Carregar os dados de estações
metro = load_estacoes("dados/GEOSAMPA-MAPA-TREM-E-METRO-SP/estacao_metro.geojson")
metro['tipo']='metro'
metro_proj = load_estacoes("dados/GEOSAMPA-MAPA-TREM-E-METRO-SP/estacao_metro_projetada.geojson")
metro_proj['tipo']='metro'
trem = load_estacoes("dados/GEOSAMPA-MAPA-TREM-E-METRO-SP/estacao_trem.geojson")
trem['tipo']='trem'
trem_proj = load_estacoes("dados/GEOSAMPA-MAPA-TREM-E-METRO-SP/estacao_trem_projetada.geojson")
trem_proj['tipo']='trem'


existente=pd.concat([metro,trem],ignore_index=True)
existente['existencia']='atual'
projetado=pd.concat([metro_proj,trem_proj],ignore_index=True)
projetado['existencia']='projetado'
all=pd.concat([existente,projetado],ignore_index=True)

all.to_parquet("dados/estacoes_metro_e_trem.parquet",index=False)

In [ ]:
gdf_resultado=gpd.sjoin_nearest(
    gdf_resultado,
    existente[['nm_estacao_metro_trem','geometry']].rename(columns={"nm_estacao_metro_trem":"nome_estacao_mais_proxima_existente"}),
    how='left',
    max_distance=2000,
    distance_col='dist_estacao_mais_proxima_existente'
).drop(columns='index_right')

In [ ]:
gdf_resultado=gpd.sjoin_nearest(
    gdf_resultado,
    projetado[['nm_estacao_metro_trem','geometry']].rename(columns={"nm_estacao_metro_trem":"nome_estacao_mais_proxima_projetado"}),
    how='left',
    max_distance=2000,
    distance_col='dist_estacao_mais_proxima_projetado'
).drop(columns='index_right')

In [ ]:


gdf_resultado=gpd.sjoin_nearest(
    gdf_resultado,
    all[['nm_estacao_metro_trem','geometry']].rename(columns={"nm_estacao_metro_trem":"nome_estacao_mais_proxima"}),
    how='left',
    max_distance=2000,
    distance_col='dist_estacao_mais_proxima'
).drop(columns='index_right')

In [ ]:
gdf_resultado.columns

Index(['stop_id', 'stop_name', 'stop_desc', 'stop_lat', 'stop_lon',
       'data_referencia', 'geometry', 'CD_SETOR', 'SITUACAO', 'CD_SIT',
       'CD_TIPO', 'AREA_KM2', 'CD_REGIAO', 'NM_REGIAO', 'CD_UF', 'NM_UF',
       'CD_MUN', 'NM_MUN', 'CD_DIST', 'NM_DIST', 'CD_SUBDIST', 'NM_SUBDIST',
       'CD_BAIRRO', 'NM_BAIRRO', 'CD_NU', 'NM_NU', 'CD_FCU', 'NM_FCU',
       'CD_AGLOM', 'NM_AGLOM', 'CD_RGINT', 'NM_RGINT', 'CD_RGI', 'NM_RGI',
       'CD_CONCURB', 'NM_CONCURB', 'v0001', 'v0002', 'v0003', 'v0004', 'v0005',
       'v0006', 'v0007', 'Regiões', 'Subprefeituras ', 'Distritos',
       'nome_estacao_mais_proxima_existente',
       'dist_estacao_mais_proxima_existente',
       'nome_estacao_mais_proxima_projetado',
       'dist_estacao_mais_proxima_projetado', 'nome_estacao_mais_proxima',
       'dist_estacao_mais_proxima'],
      dtype='object')

In [ ]:
gdf_resultado.head()

,stop_id,stop_name,stop_desc,stop_lat,stop_lon,data_referencia,geometry,CD_SETOR,SITUACAO,CD_SIT,CD_TIPO,AREA_KM2,CD_REGIAO,NM_REGIAO,CD_UF,NM_UF,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_SUBDIST,NM_SUBDIST,CD_BAIRRO,NM_BAIRRO,CD_NU,NM_NU,CD_FCU,NM_FCU,CD_AGLOM,NM_AGLOM,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,v0001,v0002,v0003,v0004,v0005,v0006,v0007,Regiões,Subprefeituras,Distritos,nome_estacao_mais_proxima_existente,dist_estacao_mais_proxima_existente,nome_estacao_mais_proxima_projetado,dist_estacao_mais_proxima_projetado,nome_estacao_mais_proxima,dist_estacao_mais_proxima
0,18848,Clínicas,None,-23.554190,-46.670723,201501,POINT (329472.152 7394135.662),355030845000385,Urbana,1,0,0.197493,3,Sudeste,35,São Paulo,3550308,São Paulo,355030845,Jardim Paulista,35503084500,None,.,None,.,None,.,None,.,None,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,153.0,136.0,136.0,0.0,1.7,0.1538,91.0,Oeste,Pinheiros,Jardim Paulista,CLÍNICAS,9.610827,OSCAR FREIRE,682.296946,CLÍNICAS,9.610827
1,18849,Vila Madalena,None,-23.546720,-46.690738,201501,POINT (327419.095 7394938.935),355030862000250,Urbana,1,0,0.040169,3,Sudeste,35,São Paulo,3550308,São Paulo,355030862,Pinheiros,35503086200,None,.,None,.,None,.,None,.,None,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,194.0,270.0,269.0,1.0,1.5,0.1395,129.0,Oeste,Pinheiros,Pinheiros,VILA MADALENA,38.371089,GIRASSOL,752.351056,VILA MADALENA,38.371089
2,18850,Consolação,None,-23.557501,-46.660872,201501,POINT (330482.069 7393780.692),355030845000289,Urbana,1,0,0.036614,3,Sudeste,35,São Paulo,3550308,São Paulo,355030845,Jardim Paulista,35503084500,None,.,None,.,None,.,None,.,None,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,229.0,133.0,133.0,0.0,2.0,0.1875,112.0,Oeste,Pinheiros,Jardim Paulista,CONSOLAÇÃO,44.537307,OSCAR FREIRE,975.731020,CONSOLAÇÃO,44.537307
3,18851,Conceição,None,-23.636214,-46.641141,201501,POINT (332596.277 7385087.28),355030838000513,Urbana,1,0,0.032082,3,Sudeste,35,São Paulo,3550308,São Paulo,355030838,Jabaquara,35503083800,None,.,None,.,None,.,None,.,None,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,271.0,137.0,137.0,0.0,2.2,0.0000,122.0,Sul,Jabaquara,Jabaquara,CONCEIÇÃO,94.642393,JABAQUARA,1267.437148,CONCEIÇÃO,94.642393
4,18852,Jabaquara,None,-23.645754,-46.642084,201501,POINT (332512.214 7384029.716),355030838000787,Urbana,1,4,0.295039,3,Sudeste,35,São Paulo,3550308,São Paulo,355030838,Jabaquara,35503083800,None,.,None,.,None,.,None,.,None,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,Sul,Jabaquara,Jabaquara,JABAQUARA,137.691339,JABAQUARA,246.268406,JABAQUARA,137.691339
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2422922,960011508,"R. Caminho Do Engenho, 417",Ref.: R Bartolomeu Bandinelli/ R Bom Jesus Dos...,-23.596402,-46.743436,202503,POINT (322105.832 7389372.573),355030894000276,Urbana,1,0,0.030568,3,Sudeste,35,São Paulo,3550308,São Paulo,355030894,Vila Sônia,35503089400,None,.,None,.,None,.,None,.,None,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,148.0,73.0,73.0,0.0,2.4,0.0164,61.0,Oeste,Butantã,Vila Sônia,VILA SÔNIA,945.440192,CHÁCARA DO JÓQUEI,570.452901,CHÁCARA DO JÓQUEI,570.452901
2422923,960011509,"Av. Dr. Guilherme Dumont Villares, 50",Ref.: R Alfredo Mendes Da Silva/ Av Professor ...,-23.604681,-46.747021,202503,POINT (321751.149 7388451.265),355030894000031,Urbana,1,0,0.088772,3,Sudeste,35,São Paulo,3550308,São Paulo,355030894,Vila Sônia,35503089400,None,.,None,.,None,.,None,.,None,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,308.0,130.0,129.0,1.0,2.5,0.2202,109.0,Oeste,Butantã,Vila Sônia,VILA SÔNIA,1767.124530,CHÁCARA DO JÓQUEI,479.632313,CHÁCARA DO JÓQUEI,479.632313
2422924,960011510,"R. Alfredo Mendes Da Silva, 499",Ref.: R Cedro/ Av Doutor Guilherme Dumont Vill...,-23.604420,-46.748357,202503,POINT (321614.455 7

In [15]:
gdf_resultado.to_parquet("dados/paradas_historico.parquet",index=False)